## Installation & Setup

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
! git clone https://github.com/Yogesh31Hasabe/NCSU-CSC_591-RealTime_AI_and_Machine_Learning_Systems-CourseProject-LPRNet.git

Cloning into 'NCSU-CSC_591-RealTime_AI_and_Machine_Learning_Systems-CourseProject-LPRNet'...
remote: Enumerating objects: 1019, done.
remote: Counting objects: 100% (1019/1019), done.
remote: Compressing objects: 100% (1015/1015), done.
remote: Total 1019 (delta 1), reused 1015 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (1019/1019), 17.81 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
cd NCSU-CSC_591-RealTime_AI_and_Machine_Learning_Systems-CourseProject-LPRNet

/content/NCSU-CSC_591-RealTime_AI_and_Machine_Learning_Systems-CourseProject-LPRNet


In [4]:
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os
import copy
from types import SimpleNamespace
from collections import OrderedDict
import torch.nn.utils.prune as prune

In [5]:
args = {
    'img_size': [94, 24],
    'test_img_dirs': "./data/test",
    'dropout_rate': 0,
    'lpr_max_len': 7,
    'test_batch_size': 100,
    'phase_train': False,
    'num_workers': 2,
    'cuda': False,
    'show': False,
    'pretrained_model': './weights/Final_LPRNet_model.pth'
}

args = SimpleNamespace(**args)

In [6]:
lprnet = build_lprnet(
    lpr_max_len=args.lpr_max_len,
    phase=args.phase_train,
    class_num=len(CHARS),
    dropout_rate=args.dropout_rate
)

device = torch.device("cuda:0" if args.cuda else "cpu")
lprnet.to(device)
print("Network Built Successfully !! \n")
print(lprnet)

Network Built Successfully !! 

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequenti

In [7]:
# load pretrained model
if args.pretrained_model:
    lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=torch.device('cpu')))
    print("Pretrained Model loaded successfully !!")
else:
    print("[Error] Can't found pretrained mode, please check!")

Pretrained Model loaded successfully !!


<ipython-input-7-c91668bc6022>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=torch.device('cpu')))


## Baseline Model Accuracy

In [8]:
! python test_LPRNet.py

Successful to build network!
/content/NCSU-CSC_591-RealTime_AI_and_Machine_Learning_Systems-CourseProject-LPRNet/test_LPRNet.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

## Pruning (Unstructured Prunning)

In [9]:
lprnet_pruned = lprnet
module = lprnet_pruned.backbone
print(lprnet.state_dict().keys())

odict_keys(['backbone.0.weight', 'backbone.0.bias', 'backbone.1.weight', 'backbone.1.bias', 'backbone.1.running_mean', 'backbone.1.running_var', 'backbone.1.num_batches_tracked', 'backbone.4.block.0.weight', 'backbone.4.block.0.bias', 'backbone.4.block.2.weight', 'backbone.4.block.2.bias', 'backbone.4.block.4.weight', 'backbone.4.block.4.bias', 'backbone.4.block.6.weight', 'backbone.4.block.6.bias', 'backbone.5.weight', 'backbone.5.bias', 'backbone.5.running_mean', 'backbone.5.running_var', 'backbone.5.num_batches_tracked', 'backbone.8.block.0.weight', 'backbone.8.block.0.bias', 'backbone.8.block.2.weight', 'backbone.8.block.2.bias', 'backbone.8.block.4.weight', 'backbone.8.block.4.bias', 'backbone.8.block.6.weight', 'backbone.8.block.6.bias', 'backbone.9.weight', 'backbone.9.bias', 'backbone.9.running_mean', 'backbone.9.running_var', 'backbone.9.num_batches_tracked', 'backbone.11.block.0.weight', 'backbone.11.block.0.bias', 'backbone.11.block.2.weight', 'backbone.11.block.2.bias', 'ba

In [10]:
parameters_to_prune = []

for name, module in lprnet_pruned.named_modules():
    if isinstance(module, nn.Conv2d):
        parameters_to_prune.append((module, 'weight'))

In [11]:
parameters_to_prune

[(Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)), 'weight'),
 (Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 4), stride=(1, 1)), 'weight'),
 (Co

In [12]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.9,
)

for module, _ in parameters_to_prune:
    prune.remove(module, 'weight')

## Save Model Weights

In [ ]:
torch.save(lprnet_pruned.state_dict(), './weights/lprnet_model_optimization_pruning.pth')

## Test Function - 1 : Size

In [13]:
def print_size_of_model(model, model_name="Model"):
    """
    Save the model temporarily to measure its size on disk and print the size.
    Args:
        model (torch.nn.Module): The model to evaluate.
        model_name (str): Name of the model for reference in output.
    """
    torch.save(model.state_dict(), "temp_delme.p")
    model_size_kb = os.path.getsize("temp_delme.p") / 1e3
    print(f"{model_name} Size (KB): {model_size_kb:.2f}")
    os.remove("temp_delme.p")


In [14]:
print('Size of the model before pruning:')
print_size_of_model(lprnet, model_name="Original Model")
print("\n")

print('Size of the model after pruning:')
print_size_of_model(lprnet_pruned, model_name="Pruned Model")
print("\n")

Size of the model before pruning:
Original Model Size (KB): 1816.74


Size of the model after pruning:
Pruned Model Size (KB): 1816.74




## Test Function - 2 : Accuracy & Speed

In [15]:
def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for sample in batch:
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = torch.tensor(labels, dtype=torch.float32)
    return torch.stack(imgs, dim=0), labels, lengths


def Greedy_Decode_Eval(Net, datasets, args):
    # Net.eval()

    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets,
                                      args.test_batch_size,
                                      shuffle=True,
                                      num_workers=args.num_workers,
                                      collate_fn=collate_fn))

    Tp, Tn_1, Tn_2 = 0, 0, 0
    t1 = time.time()

    for _ in range(epoch_size):

        images, labels, lengths = next(batch_iterator)
        targets = []
        start = 0

        for length in lengths:
            targets.append(labels[start:start + length])
            start += length
        targets = np.array([target.numpy() for target in targets])


        if args.cuda:
            images = images.cuda()


        with torch.no_grad():
            prebs = Net(images)


        prebs = prebs.cpu().numpy()
        preb_labels = []

        for preb in prebs:
            preb_label = [np.argmax(preb[:, j]) for j in range(preb.shape[1])]
            no_repeat_blank_label = []
            pre_c = preb_label[0]


            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)

            for c in preb_label:
                if c == len(CHARS) - 1 or c == pre_c:
                    pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c

            preb_labels.append(no_repeat_blank_label)


        for i, label in enumerate(preb_labels):
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if np.array_equal(label, targets[i]):
                Tp += 1
            else:
                Tn_2 += 1


    total_samples = Tp + Tn_1 + Tn_2
    Acc = Tp / total_samples
    t2 = time.time()

    print(f"[Info] Test Accuracy: {Acc:.4f} [Tp:{Tp}, Tn_1:{Tn_1}, Tn_2:{Tn_2}, Total:{total_samples}]")
    print(f"[Info] Test Speed: {(t2 - t1) / len(datasets):.6f}s per sample [{len(datasets)} samples]")


def test(model):
    test_img_dirs = os.path.expanduser(args.test_img_dirs)
    test_dataset = LPRDataLoader(test_img_dirs.split(','),
                                 args.img_size,
                                 args.lpr_max_len)
    Greedy_Decode_Eval(model, test_dataset, args)

In [16]:
print('Testing the model after pruning \n')
test(lprnet_pruned)
print("\n")

Testing the model after pruning 

[Info] Test Accuracy: 0.9010 [Tp:901, Tn_1:65, Tn_2:34, Total:1000]
[Info] Test Speed: 0.031716s per sample [1000 samples]


